# Post office problem in OR-tools CP-SAT Solver
Source: http://www.hakank.org/or_tools/post_office_problem2_sat.py

From Winston 'Operations Research: Applications and Algorithms':

A post office requires a different number of full-time employees working on different days of the week [summarized below]. Union rules state that each full-time employee must work for 5 consecutive days and then receive two days off. For example, an employee who works on Monday to Friday must be off on Saturday and Sunday. The post office wants to meet its daily requirements using only full-time employees. Minimize the number of employees that must be hired.

To summarize the important information about the problem:
- Every full-time worker works for 5 consecutive days and takes 2 days off
- Day 1: 17 workers needed (Monday)
- Day 2: 13 workers needed
- Day 3: 15 workers needed
- Day 4: 19 workers needed
- Day 5: 14 workers needed
- Day 6: 16 workers needed
- Day 7: 11 workers needed (Sunday)
- The post office needs to minimize the number of employees it needs to hire to meet its demand.
  
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Solver Max notes

The original model appears to be wrong. That is, it had a constraint expressed in terms of:

j != (i + 5) % n and j != (i + 6) % n

That constraint is forward-looking by 5 and 6 days. For example, it means that Monday excludes workers who start on Saturday and Sunday. However, it should exclude workers who start on the next two days, that is:

j != (i + 1) % n and j != (i + 2) % n

Given the revised constraint, the optimal solution has a total 21 people working on Friday, though only 14 are needed. To reduce this gap, we introduced another component to the objective function: the maximum total number of workers on any day. This also requires a variable to represent that maximum, and a constraint that the number of workers on a day is <= max_workers.

With weights of 1.0 and 0.1 for the objectives, the solution has the same cost, but a better distrubution of workers (max is 19 rather than 21).

With weights of 0.0 and 1.0 for the objectives, we get a slightly higher cost (15,500 rather than 15,300), though the number of workers remains at 23.

The output has been extended, to provide additional information about the solution.

In [1]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp
import math, sys
from cp_sat_utils import scalar_product

In [2]:
def main():
    model = cp.CpModel()
   
    n = 7   # days 0..6, monday 0
    days = list(range(n))
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    need = [17, 13, 15, 19, 14, 16, 11]
    cost = [500, 600, 800, 800, 800, 800, 700]   # Total cost for the 5 day schedule. Base cost per day is 100. Working saturday is 100 extra. Working sunday is 200 extra.
    x = [model.NewIntVar(0, 100, 'x[%i]' % i) for i in days]   # No. of workers starting at day i
    total_cost = model.NewIntVar(0, 20000, 'total_cost')
    num_workers = model.NewIntVar(0, 100, 'num_workers')
    max_workers = model.NewIntVar(0, 100, 'max_workers')

    scalar_product(model, x, cost, total_cost)
    model.Add(num_workers == sum(x))

    for i in days:
        model.Add(sum([x[j] for j in days if j != (i + 1) % n and j != (i + 2) % n]) >= need[i])
        model.Add(sum([x[j] for j in days if j != (i + 1) % n and j != (i + 2) % n]) <= max_workers)

    model.Minimize(total_cost * 1.0 + max_workers * 0.1)

    solver = cp.CpSolver()
    status = solver.Solve(model)

    if status == cp.OPTIMAL:
        print(f'Workers:    {solver.Value(num_workers):8,}')
        print(f'Total cost: {solver.Value(total_cost):8,}')
        print()
        #print('x:', [solver.Value(x[i]) for i in days])
        print('Day         Start  Total   Need')
        print('-------------------------------')
        for i in days:
            print(f'{day_names[i]:10} {solver.Value(x[i]):6,} {sum([solver.Value(x[j]) for j in days if j != (i + 1) % n and j != (i + 2) % n]):6,} {solver.Value(need[i]):6,} ')
        
    print()
    print('Solver statistics:')
    print('---------------------------')
    print(f'Number of conflicts: {solver.NumConflicts():6,}')
    print(f'Number of branches:  {solver.NumBranches():6,}')
    print(f'\nWall time: {solver.WallTime():6,.3f} seconds')

In [3]:
if __name__ == '__main__':
    main()

Workers:          23
Total cost:   15,300

Day         Start  Total   Need
-------------------------------
Monday          6     17     17 
Tuesday         6     17     13 
Wednesday       0     16     15 
Thursday        6     19     19 
Friday          1     19     14 
Saturday        3     16     16 
Sunday          1     11     11 

Solver statistics:
---------------------------
Number of conflicts:    247
Number of branches:     261

Wall time:  0.026 seconds
